Instala e importa bibliotecas e libera acesso ao drive

In [ ]:
%pip install requests
%pip install geopandas
!pip install rasterio

In [2]:
import requests
import folium
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import numpy as np
import math
import geopandas as gpd
import rasterio
from shapely.geometry import Point, LineString, Polygon
import matplotlib as mpl
from scipy.spatial import Voronoi
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Configura o pandas para mostrar todas as colunas do dataframe
pd.options.display.max_columns = None


Mounted at /content/drive


Função manipula o DataFrame

In [31]:
def prepararDF(df, codigo):
  # Muda o tipo da coluna 'Data' para datetime (string --> datetime)
  df['Data'] = df['DataHora'][:10]
  df['Data'] = pd.to_datetime(df['Data'], format='%Y/%m/%d')

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice
  df.set_index(['Data','NivelConsistencia'], inplace=True)

  # Exclui colunas desnecessárias do dataframe
  listColumnsDrop = []
  for i in range(77):
    if i > 44:
      listColumnsDrop += [i]
  listColumnsDrop += [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
  df = df.drop(df.columns[listColumnsDrop], axis=1)
  
  # Avalia se existe uma mesma data com diferentes níveis de consistência
  # Classificar o dataframe por data
  df.sort_values("Data", inplace=True)

  # Reseta os índices para passar "Data" como coluna e utilizar a mesma para apagar as linhas duplicadas
  df = df.reset_index()

  # apagas as linhas duplicadas, dando preferência ao dado consistido ('NivelConsistencia' = 2)
  df = df.drop_duplicates(subset='Data', keep='last', inplace=False)

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice
  df.set_index(['Data','NivelConsistencia'], inplace=True)

  # transforma as colunas de chuva em linhas com as respectivas datas
  # Este processo transforma todas as colunas em linhas, gerando um problema para os meses que possuem menos de 31 dias
  # Dessa forma, o dia 1 de um mês armazenava a chuva do dia 1 e a chuva do dia 31 (linha duplicada com valor NaN)
  # Correção explicada durante o processo:

  # usa a função .melt() para transformar todas as colunas em linhas. São mantidas as colunas ['Data', 'NivelConsistencia']
  # São criadas duas novas colunas: 'Dia' que recebe o nome da coluna que virou linha e 'Chuva' que recebe o valor que era armazenado na coluna
  df = df.reset_index().melt(id_vars=['Data', 'NivelConsistencia'], var_name='Dia', value_name='Chuva')

  # Fiz uma cópia da minha data em formato de texto
  df['Data1']=df['Data'].astype(str)

  # Apaguei os últimos 2 caracteres, assim ficou somente o texto com %Y-%m- ('1989-11-')
  df['Data1'] = df['Data1'].str[:-2]

  # Fiz uma cópia da coluna 'Dia' (exe: 'Chuva03'), a 'função .str.extract('(\d+)', expand=False)' deixa apenas os números
  # (então a coluna 'Dia1' armazena a string '03') ('Chuva03' --> '03')  
  df['Dia1'] = df['Dia'].str.extract('(\d+)', expand=False)
  
  # praticamente a mesma coisa da linha anterior, mudando apenas que substitui a própria coluna e o formato é int ('Chuva03' --> 3)
  df['Dia'] = df['Dia'].str.extract('(\d+)', expand=False).astype(int)

  # Como foram criadas várias linhas, tem vários índices repetidos da data, esta função soma os dias correspondente
  # aos da chuva guardado na coluna 'Dia' na data do índice (aqui é gerado o erro dos 31 dias)
  df['Data'] = df.apply(lambda x: x['Data'] + pd.DateOffset(days=x['Dia']-1), axis=1)

  # Aqui é feita uma concatenação do tipo '1989-11-'+'03' formando uma data.
  # Mas aqui também forma datas como '1989-02-31'
  df['Data1'] = df['Data1'] + df['Dia1']

  # A maioria das datas das colunas 'Data' e 'Data1' serão iguais, mas datas como '1989-02-31'
  # vão gerar valor NaT pois é um erro (exatamente nas linhas duplicadas)
  df['Data1'] = pd.to_datetime(df['Data1'], format='%Y-%m-%d', errors='coerce')

  # Aqui é apagada todas as linhas em que o erro NaT aparece na coluna 'Data1' (removendo os valores duplicados)
  df.dropna(inplace=True, subset=['Data1'])

  # reorganiza as colunas
  df = df[['Data', 'NivelConsistencia', 'Chuva']]

  # Classificar o dataframe por data
  df.sort_values("Data", inplace=True)

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice novamente
  #df.set_index(['Data'], inplace=True)
  df['Codigo'] = codigo
  return df

Obter dados de estações pluviométricas(2) de uma Sub-Bacia(39) através de API.

In [ ]:
#https://www.hashtagtreinamentos.com/o-que-e-uma-api-python?gclid=Cj0KCQiAx6ugBhCcARIsAGNmMbi3iOmaN0FmKdQR3RaduU3zTQfIQvKN3enrln7Xa2S7UW3zTftBECIaApFJEALw_wcB
#https://dadosaocubo.com/ingestao-de-dados-via-api-com-python/#:~:text=Requisi%C3%A7%C3%A3o%20de%20Dados%20da%20API,vamos%20deixar%20vazio%20neste%20momento).
#https://dadosabertos.ana.gov.br/documents/ae318ebacb4b41cda37fbdd82125078b/explore
#https://www.nylas.com/blog/use-python-requests-module-rest-apis/
#https://pandas.pydata.org/docs/reference/io.html#json

Função que usa API para obter todas as estações da sub-bacia

In [32]:
def estacoesSubBaciaAPI(cod):
  url = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroInventario'
  params = {
      'codEstDE': '',
      'codEstATE': '',
      'tpEst': '2',
      'nmEst': '',
      'nmRio': '',
      'codSubBacia': cod,
      'codBacia': '',
      'nmMunicipio': '',
      'nmEstado': '',
      'sgResp': '',
      'sgOper': '',
      'telemetrica': ''
  }
  # Realiza a requisição GET para a URL fornecida, passando os parâmetros
  response = requests.get(url, params=params)

  xml = response.text

  # Lê o XML e converte em um DataFrame utilizando o método read_xml do pandas
  df = pd.read_xml(xml,
                  xpath="//Table",
                  namespaces={"xs": "http://www.w3.org/2001/XMLSchema"})

  # Seleciona apenas as colunas desejadas no DataFrame df
  df = df[['Codigo', 'TipoEstacaoTelemetrica', 'PeriodoPluviometroInicio', 'PeriodoPluviometroFim', 'PeriodoTelemetricaInicio', 'PeriodoTelemetricaFim', 'MunicipioCodigo', 'nmMunicipio', 'Latitude', 'Longitude']]

  # Define a coluna 'Codigo' como índice do DataFrame df
  df.set_index(['Codigo'], inplace=True)

  # Converte as colunas de datas para o formato datetime
  df['PeriodoPluviometroInicio'] = pd.to_datetime(df['PeriodoPluviometroInicio'], format='%Y-%m-%d %H:%M:%S')
  df['PeriodoPluviometroFim'] = pd.to_datetime(df['PeriodoPluviometroFim'], format='%Y-%m-%d %H:%M:%S')
  df['PeriodoTelemetricaInicio'] = pd.to_datetime(df['PeriodoTelemetricaInicio'], format='%Y-%m-%d %H:%M:%S')
  df['PeriodoTelemetricaFim'] = pd.to_datetime(df['PeriodoTelemetricaFim'], format='%Y-%m-%d %H:%M:%S')
  return df

Função para ler de arquivo a geometria da sub-bacia

In [33]:
def carregaGPKG(caminho_para_pasta, caminho_para_arquivo, bacia):
  # Carrega os polígonos das bacias a partir do arquivo usando o geopandas
  bacias = gpd.read_file(f'{caminho_para_pasta}{caminho_para_arquivo}')

  # Filtra o dataframe de bacias para manter apenas as bacias com o valor '7596' na coluna 'wts_cd_pfafstetterbasin'
  bacia_filtrada = bacias[bacias['wts_cd_pfafstetterbasin'] == bacia]

  # Cria buffer
  bacia_filtrada_buffer = bacia_filtrada.buffer(0.1) #Não sei que unidade de medida é essa do buffer
  return bacia_filtrada_buffer

Função para gerar gráfico e retornar estações dentro da bacia

In [34]:
def graficoEstacoesBacia(bacia_filtrada_buffer,df):
  # Faz um recorte entre estações e bacia com buffer
  GDF = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
  GDF_Filtrado = GDF[GDF.within(bacia_filtrada_buffer.geometry.iloc[0])]
  DF_Filtrado = GDF_Filtrado.drop('geometry', axis=1).copy()

  # criar mapa
  mapa = folium.Map(location=[GDF_Filtrado['Latitude'].mean(), GDF_Filtrado['Longitude'].mean()], zoom_start=9)

  # adicionar marcadores para cada coordenada
  for index, row in GDF_Filtrado.iterrows():
      # definir cor do marcador com base no valor da coluna "TipoEstacaoTelemetrica"
      if row['TipoEstacaoTelemetrica'] == 1:
          cor = 'red'
          texto = 'C/Telemetria'
      else:
          cor = 'blue'
          texto = 'S/Telemetria'
      folium.Marker(location=[row.geometry.y, row.geometry.x], popup=f'Código:{index}'+'\n'+f'{texto}', icon=folium.Icon(color=cor)).add_to(mapa)

  # adicionar polígonos como camada no mapa

  folium.GeoJson(bacia_filtrada_buffer).add_to(mapa)

  # exibir mapa
  return mapa, DF_Filtrado

Função para obter dados de estações

In [43]:
def estacoesAPI(DF_Filtrado):
    # API para solicitar dados de estações sem telemetria (As com telemetria estão dando erro)
    dfRequest = DF_Filtrado[DF_Filtrado['TipoEstacaoTelemetrica'] == 0]
    df_sum_NaN = pd.DataFrame(columns=['Estacao', 'SumNaN']) 
    frames = []
    for estacao in list(dfRequest.index):
        url = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica'
        params = {
            'codEstacao': estacao,
            'dataInicio': '',
            'dataFim': '',
            'tipoDados': '2',
            'nivelConsistencia': '',
        }
        try:
            response = requests.get(url, params=params)
            xml = response.text
            df1 = pd.read_xml(xml, xpath="//SerieHistorica", namespaces={"xs": "http://www.w3.org/2001/XMLSchema"})
        except ValueError:
            pass
        else:
            response = requests.get(url, params=params)
            xml = response.text
            df1 = pd.read_xml(xml, xpath="//SerieHistorica", namespaces={"xs": "http://www.w3.org/2001/XMLSchema"})
            df1 = prepararDF(df1, estacao)
            num_nans = df1['Chuva'].isna().sum()
            df_sum_NaN = df_sum_NaN.append({'Estacao': estacao, 'SumNaN': num_nans}, ignore_index=True)
            frames.append(df1)
    dfEstacoesST = pd.concat(frames)
    dfEstacoesST = dfEstacoesST.dropna(subset=['Chuva'])
    print(f'Foram carregadas {len(frames)} estações')
    return dfEstacoesST, df_sum_NaN

Função recebe dados de estações em DataFrame e cria gráfico de GANTT

In [44]:
def graficoGANTT(df1):
  df1 = df1.sort_values(["Codigo", "Data"])

  grupos = []
  seq_datas = []

  for codigo, grupo in df1.groupby("Codigo"):
      for i, data in enumerate(grupo["Data"]):
          if i == 0:
              seq_datas.append([data])
          else:
              if (data - grupo["Data"].iloc[i-1]) == timedelta(days=1):
                  seq_datas[-1].append(data)
              else:
                  seq_datas.append([data])
      for seq in seq_datas:
          grupos.append({
              "Task": codigo,
              "Start": seq[0],
              "Finish": seq[-1]
          })
      seq_datas = []

  df_grupos_GANTT = pd.DataFrame(grupos)

  fig = ff.create_gantt(df_grupos_GANTT, show_colorbar=False,
                        group_tasks=True)
  fig.show()

Chama as funções.
Observe que no mapa, os marcadores vermelhos são estações que possuem telemetria e os marcadores azuis não possuem telemetria. Clicando no marcador é mostrado o código da estação e a informação sobre telemetria.

In [48]:
# Carrega estações da bacia 39
cod = '39'
df = estacoesSubBaciaAPI(cod)

# Define o caminho
caminho_para_pasta = '/content/drive/MyDrive/ICD/Colab/Alexandre'
caminho_para_arquivo = '/geoft_bho_ach_otto_nivel_04.gpkg'
# Filtro
bacia = '7596'

# Carrega geometria da bacia com buffer
bacia_filtrada_buffer = carregaGPKG(caminho_para_pasta, caminho_para_arquivo, bacia)

# Cria DataFrame com todas as estações dentro da bacia
DF_Filtrado = graficoEstacoesBacia(bacia_filtrada_buffer,df)[1]

# Chama mapa apresentando bacia e estações
graficoEstacoesBacia(bacia_filtrada_buffer,df)[0]

Chama as funções que obtem dados das estações através de API e grafico de GANTT

In [52]:
dados = estacoesAPI(DF_Filtrado)
graficoGANTT(dados[0])

Número de falhas nas leituras das estações

In [53]:
dados[1]

,Estacao,SumNaN
0,835073,0
1,836008,0
2,836009,0
3,836013,0
4,836015,0
5,836016,0
6,836017,0
7,836020,0
8,836021,0
9,836022,0


Analisando o gráfico de GANTT é possível notar que a disponibilidade de dados ao longo do tempo está bastante pulverizada e com períodos curtos. Avaliando rápidamente as falhas nas leituras das estações, nota-se um comportamento estranho pois a maioria não apresenta falha (O que não justifica a pulverização dos dados apresentados no gráfico de GANTT) e os que apresentam falha é um ano inteiro, levando a crer que a um erro da ANA ao fornecer os dados via API. Vale levar em consideração também que não foi possível solicitar dados de estações telemétricas.